In [13]:
import pandas as pd
import requests
import os
import shutil

#### **Notice**  
1. For data of volume, **Bitcon** data is incomplete, so we chose to eliminate the Bitcon in the volume section, and chose to plot the volume graph in the next place in the tvl order sort, i.e., the **Polygon**.  
2. For the chainname obtained through the tvl sort is not completely correct, in the api of the fee and volume there are some chains can not get the data, we are here to eliminate these data.  
3. We need data of 150 days to calculate the z-score to recognize abnormal values. We only plot data of 120 days.

In [ ]:

# you can get the chain list straihgt from the API
url = 'https://api.llama.fi/v2/chains'
response = requests.get(url)
if response.status_code == 200: # data was fetched successfully
    data = response.json()
    df = pd.DataFrame(data)
else:
    raise Exception('Error fetching data from DeFiLlama API')

dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
dex_all_chains_response = requests.get(dex_all_chains_url)

dex_all_chains_data = dex_all_chains_response.json()
dex_all_chains = set(dex_all_chains_data.get('allChains', []))


fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
fees_all_chains_response = requests.get(fees_all_chains_url)

fees_all_chains_data = fees_all_chains_response.json()
fees_all_chains = set(fees_all_chains_data.get('allChains', []))

common_chains = dex_all_chains.intersection(fees_all_chains) # get the common chains between the two sets

df = df[['name', 'tvl']]
df = df[df['name'].isin(common_chains)]

# Sort the data to get the top 10 chains by TVL
df = df.sort_values(by='tvl', ascending=False).head(11)
print(df)

base_folder = './data'
fee_folder = os.path.join(base_folder, 'fee')
tvl_folder = os.path.join(base_folder, 'tvl')
volume_folder = os.path.join(base_folder, 'volume')
price_folder = os.path.join(base_folder, 'price')
price_folder_USD = os.path.join(base_folder, 'price_USD')

for folder in [fee_folder, tvl_folder, volume_folder, price_folder, price_folder_USD]:
    if os.path.exists(folder):
        shutil.rmtree(folder)  # delete the folder and its contents
    os.makedirs(folder)

         name           tvl
17   Ethereum  7.604085e+10
47     Solana  9.257967e+09
92       Tron  9.088107e+09
7         BSC  5.827424e+09
9        Base  3.830352e+09
54    Bitcoin  3.747181e+09
16   Arbitrum  3.331686e+09
12  Avalanche  1.596518e+09
95        Sui  1.596115e+09
58      Aptos  1.254248e+09
18    Polygon  1.176373e+09


In [7]:
chain_order = df['name'].tolist()
chain_index = {chain: str(index + 1).zfill(2) for index, chain in enumerate(chain_order)}
# give chainname an index so we can plot in order


def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
            df['date'] = pd.to_datetime(df['date'], unit='s')
            csv_filename = os.path.join(tvl_folder, f"{chain_index[chain_name]}_{chain_name}_{metric}.csv")
            df = df[['date', metric]].tail(150)
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            df = df[['date', metric]].tail(150)
            csv_filename = os.path.join(volume_folder, f"{chain_index[chain_name]}_{chain_name}_{metric}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            df = df[['date', metric]].tail(150)
            csv_filename = os.path.join(fee_folder, f"{chain_index[chain_name]}_{chain_name}_{metric}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    else:
        raise ValueError('Invalid metric specified')

In [ ]:
import sys
import os
import getpass
# find a way to use relative path, so that the function in dynamoUtil can be run in any machine
src_path = r'C:\Users\YuweiCao\Documents\GitHub\char-python-common\src'

if src_path not in sys.path:
    sys.path.append(src_path)

USER = getpass.getuser()
REPO_PATH = r'C:\Users\YuweiCao\Documents\GitHub\char-python-common'

sys.path.append(REPO_PATH)
sys.path.append(REPO_PATH + '/src')

from dynamoUtil import price_bar_query
from datetime import datetime, timedelta

# mapping = {
#     'Ethereum': 'ETHUSDT-OKX-00000000-FUT',
#     'Tron': 'TRXUSDT-OKX-00000000-FUT',
#     'Solana': 'SOLUSDT-OKX-00000000-FUT',
#     'BSC': 'BNBUSDT-OKX-00000000-FUT',
#     'Base': 'BASEUSDT-OKX-00000000-FUT',
#     'Bitcoin': 'BTCUSDT-OKX-00000000-FUT',
#     'Arbitrum': 'ARBUSDT-OKX-00000000-FUT',
#     'Avalanche': 'AVAXUSDT-OKX-00000000-FUT',
#     'Sui': 'SUIUSDT-OKX-00000000-FUT',
#     'Aptos': 'APTUSDT-OKX-00000000-FUT',
#     'Polygon': 'MATICUSDT-OKX-00000000-FUT'
# }
mapping = {
    'Ethereum': 'ETHUSD-OKX-00000000-FUT',
    'Tron': 'TRXUSD-OKX-00000000-FUT',
    'Solana': 'SOLUSD-OKX-00000000-FUT',
    'BSC': 'BNBUSD-OKX-00000000-FUT',
    'Base': 'BASEUSD-OKX-00000000-FUT',
    'Bitcoin': 'BTCUSD-OKX-00000000-FUT',
    'Arbitrum': 'ARBUSD-OKX-00000000-FUT',
    'Avalanche': 'AVAXUSD-OKX-00000000-FUT',
    'Sui': 'SUIUSD-OKX-00000000-FUT',
    'Aptos': 'APTUSD-OKX-00000000-FUT',
    'Polygon': 'MATICUSD-OKX-00000000-FUT'
}


# give df the mapping of the code
'''
    这里有个问题，我用的是比较生硬的map，如果有新的chain，就需要手动添加，这个可以通过API来解决？
'''

def fetch_price_data(chain_name):
    today = datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
    start_date = today - timedelta(days=120)

    today_str = today.strftime('%Y-%m-%d-%H-%M')
    start_date_str = start_date.strftime('%Y-%m-%d-%H-%M')
    
    instrument_code = mapping.get(chain_name)

    price_data = price_bar_query(instrument_code + '|1440', start_date_str, today_str, 'live')
    price_df = pd.DataFrame(price_data)
    print(f"Columns in DataFrame for {chain_name}: {price_df.columns}") 
    print(price_df.head())
    
    if price_df.empty:
        print(f"No valid data found for {chain_name}. Skipping.")
        return

    if 'timestamp' not in price_df.columns or 'close' not in price_df.columns:
        print(f"Required columns 'timestamp' or 'close' not found in the data for {chain_name}. Skipping.")
        return

    selected_columns = price_df[['timestamp', 'close']].copy()

    selected_columns['timestamp'] = pd.to_datetime(selected_columns['timestamp']).dt.date

    selected_columns.rename(columns={'timestamp': 'date', 'close': 'price'}, inplace=True)

    csv_filename = os.path.join(price_folder, f"{chain_index[chain_name]}_{chain_name}_price.csv")
    # csv_filename = os.path.join(price_folder_USD, f"{chain_index[chain_name]}_{chain_name}_price_USD.csv")
    selected_columns.to_csv(csv_filename, index=False)
    print(f"Saved price data for {chain_name} to {csv_filename}")


In [16]:
# for chain_name in df['name'].head(10):
#     for metric in ['tvl', 'fee']:
#         fetch_historical_data(chain_name, metric)
        
# for chain_name in df['name']:
#     if chain_name.lower() == 'bitcoin':
#         print(f"Skipping {chain_name} for {metric}")
#         continue # Skip Bitcoin as its volume data is incomplete
#     metric = 'volume'
#     fetch_historical_data(chain_name, metric)
    
for chain_name in df['name']:
    fetch_price_data(chain_name)

Columns in DataFrame for Ethereum: Index(['low', 'updatedAt', 'timestamp', 'open', 'volume', 'high', 'close',
       'instrument'],
      dtype='object')
       low      updatedAt  timestamp     open    volume     high    close  \
0  2351.82  1723247965225 2024-08-08  2353.11  90893157     2727  2684.57   
1  2553.63  1723334123101 2024-08-09  2667.34  68323488     2709     2600   
2  2581.67  1723420695651 2024-08-10   2585.6  20589026  2643.99   2610.6   
3  2540.31  1723507033437 2024-08-11   2625.8  48528524  2720.79  2556.59   
4  2516.82  1723593342906 2024-08-12  2538.08  70117475  2751.99  2722.66   

                instrument  
0  ETHUSD-OKX-00000000-FUT  
1  ETHUSD-OKX-00000000-FUT  
2  ETHUSD-OKX-00000000-FUT  
3  ETHUSD-OKX-00000000-FUT  
4  ETHUSD-OKX-00000000-FUT  
Saved price data for Ethereum to ./data\price_USD\01_Ethereum_price_USD.csv
Columns in DataFrame for Solana: Index(['low', 'updatedAt', 'timestamp', 'open', 'volume', 'high', 'close',
       'instrument'],
   

In [ ]:
base_url = "https://data-api.cryptocompare.com/futures/v1/historical/open-interest/days"
params_trx = {
    'market': 'okex', 
    'instrument': 'TRX-USDT-VANILLA-PERPETUAL', 
    'groups': 'OHLC',  
    'limit': 120,  
    'aggregate': 1,  
    'fill': 'true',  
    'apply_mapping': 'true',  
    'api_key': 'YOUR_API_KEY'  
}

response_trx = requests.get(base_url, params=params_trx)

if response_trx.status_code == 200:
    data = response_trx.json()

    if 'Data' in data and isinstance(data['Data'], list):
        data_list = data['Data']
        
        extracted_data = [{'timestamp': entry['TIMESTAMP'], 'price': entry['CLOSE_QUOTE']}
                          for entry in data_list if 'TIMESTAMP' in entry and 'CLOSE_QUOTE' in entry]
        
        df = pd.DataFrame(extracted_data)

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

        df.rename(columns={'timestamp': 'date'}, inplace=True)

        if not df.empty:
            csv_filename = os.path.join(price_folder, f"02_Tron_price.csv")
            df.to_csv(csv_filename, index=False)
            print("Data saved to 03_Tron_price.csv")
        else:
            print("No data available for TRX-USDT on OKEx (OKX).")
    else:
        print(f"Unexpected data format: {data}")
else:
    print(f"Failed to fetch data. HTTP status code: {response_trx.status_code}")

Data saved to 02_Tron_price.csv


In [12]:
base_url = "https://data-api.cryptocompare.com/futures/v1/historical/open-interest/days"
params_pol = {
    'market': 'okex', 
    'instrument': 'POL-USDT-VANILLA-PERPETUAL', 
    'groups': 'OHLC',  
    'limit': 120,  
    'aggregate': 1,  
    'fill': 'true',  
    'apply_mapping': 'true',  
    'api_key': 'YOUR_API_KEY'  
}

response_pol = requests.get(base_url, params=params_pol)

if response_pol.status_code == 200:
    data = response_pol.json()

    if 'Data' in data and isinstance(data['Data'], list):
        data_list = data['Data']
        
        extracted_data = [{'timestamp': entry['TIMESTAMP'], 'price': entry['CLOSE_QUOTE']}
                          for entry in data_list if 'TIMESTAMP' in entry and 'CLOSE_QUOTE' in entry]
        
        df = pd.DataFrame(extracted_data)

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

        df.rename(columns={'timestamp': 'date'}, inplace=True)

        if not df.empty:
            csv_filename = os.path.join(price_folder, f"11_Polygon_price.csv")
            df.to_csv(csv_filename, index=False)
            print("Data saved to 11_Polygon_price.csv")
        else:
            print("No data available for POL-USDT on OKEx (OKX).")
    else:
        print(f"Unexpected data format: {data}")
else:
    print(f"Failed to fetch data. HTTP status code: {response_pol.status_code}")

Data saved to 11_Polygon_price.csv
